Projeto de Desenvolvimento de Novos Poços de Petróleo

No contexto da indústria petrolífera, a tomada de decisão estratégica sobre o desenvolvimento de novos poços de petróleo é crucial para o sucesso e a sustentabilidade das operações. Este projeto se concentra em abordar esse desafio, fornecendo uma estrutura metodológica baseada em aprendizado de máquina para identificar as regiões mais promissoras para exploração. Utilizando dados geológicos sintéticos de três regiões distintas, o objetivo é desenvolver modelos de regressão linear capazes de prever o volume de reservas em novos poços de petróleo. Além disso, o projeto incorpora análises de lucro e riscos, empregando técnicas como bootstrapping para avaliar os potenciais retornos e as incertezas associadas a cada região. Ao final, espera-se oferecer insights valiosos para orientar decisões estratégicas na seleção das melhores áreas para investimento e desenvolvimento de poços de petróleo.

1  Importando as bibliotecas e as bases de dados

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

2  Preparação dos dados

In [ ]:
data0.info()
print()
print()
data1.info()
print()
print()
data2.info()

In [ ]:
print(data0.sample(5))
print()
print()
print(data1.sample(5))
print()
print()
print(data2.sample(5))

Os três dataframes não possuem valores ausentes. Seus valores e tipos de dados correspondem ao que se espera. A variável id corresponde ao identificador unívoco de cada poço de petróleo, como uma variável no formato string e as demais variáveis que representam as características e capacidade de cada poço como float.

3  Treinamento e teste do modelo

In [ ]:
def train_and_test_model(data):
    # Dividindo os dados em características (X) e rótulos (y)
    X = data[['f0', 'f1', 'f2']]
    y = data['product']

    # Dividindo os dados em conjunto de treinamento e conjunto de validação
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

    # Treinando o modelo de regressão linear
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Fazendo as predições no conjunto de validação
    y_pred = model.predict(X_val)

    # Calculando o REQM (Erro Quadrático Médio)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # Calculando o volume médio previsto de reservas
    mean_predicted_volume = y_pred.mean()

    return mean_predicted_volume, rmse

In [ ]:
# Para data0
mean_predicted_volume_0, rmse_0 = train_and_test_model(data0)
print("Para data0:")
print("Volume médio previsto de reservas:", mean_predicted_volume_0)
print("REQM do modelo:", rmse_0)
print()

In [ ]:
# Para data1
mean_predicted_volume_1, rmse_1 = train_and_test_model(data1)
print("Para data1:")
print("Volume médio previsto de reservas:", mean_predicted_volume_1)
print("REQM do modelo:", rmse_1)
print()

In [ ]:
# Para data2
mean_predicted_volume_2, rmse_2 = train_and_test_model(data2)
print("Para data2:")
print("Volume médio previsto de reservas:", mean_predicted_volume_2)
print("REQM do modelo:", rmse_2)

Os resultados da análise do desempenho dos modelos de regressão linear para as três regiões revelam variações significativas. No caso da região representada por data1, observa-se um volume médio previsto de reservas relativamente baixo, mas com um REQM mínimo, indicando uma precisão considerável do modelo. Em contrapartida, as regiões representadas pelos conjuntos de dados data0 e data2 apresentam volumes médios previstos de reservas mais elevados, porém, com valores de REQM maiores, sugerindo que os modelos possam ter dificuldades em capturar a variabilidade dos dados nesses casos.

4  Cálculo do lucro

In [ ]:
# Selecionando os top 200 poços de cada região
top_wells_data0 = data0.nlargest(200, 'product')
top_wells_data1 = data1.nlargest(200, 'product')
top_wells_data2 = data2.nlargest(200, 'product')

# Calculando o lucro projetado para cada região
profit_data0 = (top_wells_data0['product'].sum() * 4500) - (100000000)
profit_data1 = (top_wells_data1['product'].sum() * 4500) - (100000000)
profit_data2 = (top_wells_data2['product'].sum() * 4500) - (100000000)

# Imprimindo os resultados
print("Lucro projetado para data0:", profit_data0)
print("Lucro projetado para data1:", profit_data1)
print("Lucro projetado para data2:", profit_data2)

Os resultados indicam que o lucro projetado para a Região 0 é de aproximadamente  66.35𝑚𝑖𝑙ℎõ𝑒𝑠,𝑝𝑎𝑟𝑎𝑎𝑅𝑒𝑔𝑖ã𝑜1é𝑑𝑒𝑐𝑒𝑟𝑐𝑎𝑑𝑒
 24.15 milhões, e para a Região 2 é de aproximadamente $70.60 milhões. Esses valores representam o potencial de lucro após considerar os custos de desenvolvimento de 200 poços de petróleo em cada região.

5  Função para Calcular o Lucro

In [ ]:
# Definindo os modelos de regressão linear para cada região
model0 = LinearRegression()
model1 = LinearRegression()
model2 = LinearRegression()

# Treinando os modelos com os dados de cada região
model0.fit(data0[['f0', 'f1', 'f2']], data0['product'])
model1.fit(data1[['f0', 'f1', 'f2']], data1['product'])
model2.fit(data2[['f0', 'f1', 'f2']], data2['product'])

# Função para calcular o lucro potencial de cada região
def calculate_region_profit(region_name, data, model):
    # Fazendo predições para todos os poços na região
    X = data[['f0', 'f1', 'f2']]
    y_pred = model.predict(X)
    
    # Adicionando as predições como uma coluna ao dataframe original
    data[region_name + '_predicted_product'] = y_pred
    
    # Ordenando os poços pela predição de volume de reservas
    data_sorted = data.sort_values(by=region_name + '_predicted_product', ascending=False)
    
    # Selecionando os 200 melhores poços
    top_200_wells = data_sorted.head(200)
    
    # Calculando o volume total de reservas dos 200 melhores poços
    total_volume = top_200_wells[region_name + '_predicted_product'].sum()
    
    # Calculando o lucro total dos 200 melhores poços
    total_profit = total_volume * 4500
    
    # Verificando se o lucro total é suficiente para cobrir o investimento
    if total_profit >= 100000000:  # Orçamento de 100 milhões para 200 poços
        return total_profit, top_200_wells
    else:
        return 0, None

# Chamando a função calculate_region_profit para cada região
profit_0, top_wells_0 = calculate_region_profit('data0', data0, model0)
profit_1, top_wells_1 = calculate_region_profit('data1', data1, model1)
profit_2, top_wells_2 = calculate_region_profit('data2', data2, model2)

# Exibindo os resultados
print("Lucro potencial para data0:", profit_0)
print("Lucro potencial para data1:", profit_1)
print("Lucro potencial para data2:", profit_2)


Os resultados mostram que a região data0 possui o maior lucro potencial, seguida pela região data2 e, por último, a região data1. Isso sugere que a alocação de recursos para o desenvolvimento de poços de petróleo deve ser priorizada em data0, seguida por data2, de acordo com os modelos de regressão linear treinados para cada região.

6  Cálculo de Riscos e Lucro para Cada Região

In [ ]:
# Custo de implantação para 200 poços de petróleo (em milhões de dólares)
budget_per_well = 100000000

# Definindo e treinando os modelos para cada região
model_0 = LinearRegression().fit(data0[['f0', 'f1', 'f2']], data0['product'])
model_1 = LinearRegression().fit(data1[['f0', 'f1', 'f2']], data1['product'])
model_2 = LinearRegression().fit(data2[['f0', 'f1', 'f2']], data2['product'])

# Função para calcular o lucro projetado para cada região
def calculate_region_profit(data, model, budget_per_well):
    # Fazendo predições para todos os poços na região
    X = data[['f0', 'f1', 'f2']]
    y_pred = model.predict(X)
    data['predicted_product'] = y_pred
    
    # Ordenando os poços pela predição de volume de reservas
    data_sorted = data.sort_values(by='predicted_product', ascending=False)
    
    # Selecionando os top 200 poços
    top_200_wells = data_sorted.head(200)
    
    # Calculando o volume total de reservas dos top 200 poços
    total_volume = top_200_wells['predicted_product'].sum()
    
    # Calculando o lucro total dos top 200 poços
    total_profit = (total_volume * 4500) - budget_per_well
    
    return total_profit

# Função para realizar bootstrapping e calcular métricas
def bootstrap_and_calculate_metrics(data, model, budget_per_well, n_samples=1000):
    profits = []
    for _ in range(n_samples):
        sampled_data = data.sample(n=500, replace=True)
        profit = calculate_region_profit(sampled_data, model, budget_per_well)
        profits.append(profit)
    
    mean_profit = np.mean(profits)
    ci_lower = np.percentile(profits, 2.5)
    ci_upper = np.percentile(profits, 97.5)
    loss_risk = np.mean(np.array(profits) < 0) * 100
    
    return mean_profit, ci_lower, ci_upper, loss_risk

# Calculando os lucros e métricas para cada região usando bootstrapping
metrics_0 = bootstrap_and_calculate_metrics(data0, model_0, budget_per_well)
metrics_1 = bootstrap_and_calculate_metrics(data1, model_1, budget_per_well)
metrics_2 = bootstrap_and_calculate_metrics(data2, model_2, budget_per_well)

# Apresentando os resultados para cada região
print("Região 0:")
print("Lucro médio:", metrics_0[0])
print("Intervalo de confiança de 95%:", (metrics_0[1], metrics_0[2]))
print("Risco de prejuízo (%):", metrics_0[3])
print()

print("Região 1:")
print("Lucro médio:", metrics_1[0])
print("Intervalo de confiança de 95%:", (metrics_1[1], metrics_1[2]))
print("Risco de prejuízo (%):", metrics_1[3])
print()

print("Região 2:")
print("Lucro médio:", metrics_2[0])
print("Intervalo de confiança de 95%:", (metrics_2[1], metrics_2[2]))
print("Risco de prejuízo (%):", metrics_2[3])


Na região 0, o lucro médio foi de aproximadamente 3.57 milhões de dólares, com um intervalo de confiança de 95% entre 1.31 e 5.72 milhões de dólares, e um risco de prejuízo de 0.1%. Já na região 1, o lucro médio foi um pouco maior, cerca de 4.46 milhões de dólares, com um intervalo de confiança entre 0.42 e 8.56 milhões de dólares e um risco de prejuízo de 1.5%. Por fim, na região 2, o lucro médio foi de aproximadamente 2.89 milhões de dólares, com um intervalo de confiança entre 1.06 e 4.83 milhões de dólares e um risco de prejuízo de 0.2%.

7  Conclusões

Com base nos novos resultados obtidos com a técnica de bootstrap, podemos observar algumas diferenças significativas em relação aos resultados anteriores. Utilizar o bootstrap nos permitiu calcular com mais precisão o lucro médio esperado, bem como entender melhor o intervalo de confiança e o risco de prejuízo associado a cada região.

Dessa forma, com base nos novos resultados, podemos concluir que a região 1 apresenta o maior lucro médio projetado, embora também tenha o maior risco de prejuízo. Já as regiões 0 e 2 têm lucros médios menores, porém com riscos de prejuízo mais baixos. A escolha da região mais indicada para o desenvolvimento de poços de petróleo dependerá da tolerância ao risco e das metas de lucro da empresa.